In [1]:
import MySQLdb
import seaborn as sns
import pandas as pd

In [2]:
!ls data/data

cb_acquisitions.sql   cb_investments.sql    cb_offices.sql
cb_degrees.sql        cb_ipos.sql           cb_people.sql
cb_funding_rounds.sql cb_milestones.sql     cb_relationships.sql
cb_funds.sql          cb_objects.sql        license.txt


In [3]:
# variables for database connection
dsn_database = "p3"   
dsn_uid = "root"             
dsn_pwd = "I prefer prequels"            

In [4]:
# connect to database
conn = MySQLdb.connect(user=dsn_uid, passwd=dsn_pwd, db=dsn_database)
cursor=conn.cursor()

In [5]:
with conn as cursor:
    cursor.execute('SELECT * FROM cb_people')
    fields = [item[0] for item in cursor.description]
    result_ppl = []
    columns = tuple( [item[0] for item in cursor.description] )
    for row in cursor:
      result_ppl.append(dict(zip(columns, row)))

/Users/mitch/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:1: DeprecationWarning: context interface will be changed.  Use explicit conn.commit() or conn.rollback().
  if __name__ == '__main__':


In [6]:
print(columns)

('id', 'object_id', 'first_name', 'last_name', 'birthplace', 'affiliation_name')


In [7]:
len(result_ppl)

226709

In [8]:
result_ppl_df = pd.DataFrame(result_ppl)
result_ppl_df.sort_values(['object_id']).head(100)

,affiliation_name,birthplace,first_name,id,last_name,object_id
8,Facebook,None,Mark,9,Zuckerberg,p:10
86,Plaxo,None,Peter,87,Lester,p:100
873,Droplet Technology,None,Dr. Steven,874,E. Saunders,p:1000
8462,Music Nation,None,Neil,8463,Capel,p:10000
87039,Unaffiliated,None,Sue,87040,Pilsch,p:100000
87040,Unaffiliated,None,Keith,87041,Kurzendoerfer,p:100002
87041,CareerEncore,None,Courtney,87042,Homer,p:100006
87042,Unaffiliated,None,Eddy,87043,Badrina,p:100009
87043,Unaffiliated,None,Michael,87044,Dadashyan,p:100010
87044,Unaffiliated,None,Jeff,87045,Grell,p:100011


In [9]:
with conn as cursor:
    cursor.execute('SELECT * FROM cb_degrees')
    fields = [item[0] for item in cursor.description]
    result_deg = []
    columns = tuple( [item[0] for item in cursor.description] )
    for row in cursor:
      result_deg.append(dict(zip(columns, row)))

/Users/mitch/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:1: DeprecationWarning: context interface will be changed.  Use explicit conn.commit() or conn.rollback().
  if __name__ == '__main__':


In [10]:
print(columns)

('id', 'object_id', 'degree_type', 'subject', 'institution', 'graduated_at', 'created_at', 'updated_at')


In [11]:
len(result_deg)

109610

In [12]:
result_deg_df = pd.DataFrame(result_deg)
result_deg_df.sort_values(by=['object_id']).head(100)

,created_at,degree_type,graduated_at,id,institution,object_id,subject,updated_at
2414,2009-03-24 22:50:22,,None,2415,Harvard University,p:10,Computer Science,2013-03-28 22:26:47
83603,2013-09-23 19:57:12,MBA,2004-01-01,83604,MIT Sloan School of Management,p:100006,Strategy & Marketing,2013-09-24 03:33:14
36786,2013-05-19 09:25:59,MS,None,36787,University of Mumbai,p:100013,Physical Chemistry,2013-06-30 03:36:04
36785,2013-05-19 09:25:59,MBA,None,36786,Rutgers University,p:100013,Finance,2013-06-30 03:36:04
14433,2011-07-09 04:08:31,BS,1996-01-01,14434,"University of Maryland, College Park",p:100018,"Marketing, French",2011-07-09 12:33:02
14432,2011-07-09 04:08:31,MBA,2004-01-01,14433,"Columbia University, Business School",p:100018,"Entrepreneurship, Marketing",2011-07-09 12:33:02
14435,2011-07-09 09:10:33,BS,2010-01-01,14436,Gogte Institute of Technology,p:100031,Information Science & Engineering,2011-07-09 12:34:53
14436,2011-07-09 11:15:09,BS,2010-01-01,14437,Qingdao University of Science & Technology,p:100036,pharmaceutical,2011-07-09 12:35:29
14438,2011-07-09 14:18:44,BS,1982-01-01,14439,"Imperial College, London",p:100045,Engineering,2011-07-09 16:53:48
32493,2013-05-02 11:39:55,MBA,None,32494,New York University (NYU),p:100049,"Entertainment, Media & Technology and Corporat...",2013-05-02 11:39:55


In [13]:
with conn as cursor:
    cursor.execute('SELECT * FROM cb_funding_rounds')
    fields = [item[0] for item in cursor.description]
    result_FR = []
    columns = tuple( [item[0] for item in cursor.description] )
    for row in cursor:
      result_FR.append(dict(zip(columns, row)))

/Users/mitch/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:1: DeprecationWarning: context interface will be changed.  Use explicit conn.commit() or conn.rollback().
  if __name__ == '__main__':


In [14]:
print(columns)

('id', 'funding_round_id', 'object_id', 'funded_at', 'funding_round_type', 'funding_round_code', 'raised_amount_usd', 'raised_amount', 'raised_currency_code', 'pre_money_valuation_usd', 'pre_money_valuation', 'pre_money_currency_code', 'post_money_valuation_usd', 'post_money_valuation', 'post_money_currency_code', 'participants', 'is_first_round', 'is_last_round', 'source_url', 'source_description', 'created_by', 'created_at', 'updated_at')


In [15]:
len(result_FR)

52928

In [16]:
result_FR_df = pd.DataFrame(result_FR)
result_FR_df.head()

,created_at,created_by,funded_at,funding_round_code,funding_round_id,funding_round_type,id,is_first_round,is_last_round,object_id,...,post_money_valuation_usd,pre_money_currency_code,pre_money_valuation,pre_money_valuation_usd,raised_amount,raised_amount_usd,raised_currency_code,source_description,source_url,updated_at
0,2007-07-04 04:52:57,initial-importer,2006-12-01,b,1,series-b,1,0,0,c:4,...,None,None,None,None,8500000,8500000,USD,None,http://www.marketingvox.com/archives/2006/12/2...,2008-02-27 23:14:29
1,2007-05-27 06:08:18,initial-importer,2004-09-01,angel,2,angel,2,0,1,c:5,...,None,USD,None,None,500000,500000,USD,None,None,2013-06-28 20:07:23
2,2007-05-27 06:09:10,initial-importer,2005-05-01,a,3,series-a,3,0,0,c:5,...,None,USD,115000000,115000000,12700000,12700000,USD,Jim Breyer: Extra $500 Million Round For Faceb...,http://www.techcrunch.com/2007/11/02/jim-breye...,2013-06-28 20:07:23
3,2007-05-27 06:09:36,initial-importer,2006-04-01,b,4,series-b,4,0,0,c:5,...,None,USD,525000000,525000000,27500000,27500000,USD,Facebook Funding,http://www.facebook.com/press/info.php?factsheet,2013-06-28 20:07:24
4,2007-05-29 11:05:59,initial-importer,2006-05-01,b,5,series-b,5,0,0,c:7299,...,None,None,None,None,10500000,10500000,USD,PhotoBucket Closes $10.5M From Trinity Ventures,http://www.techcrunch.com/2006/05/14/photobuck...,2008-04-16 17:09:12


In [17]:
with conn as cursor:
    cursor.execute('SELECT * FROM cb_relationships')
    fields = [item[0] for item in cursor.description]
    result_rel = []
    columns = tuple( [item[0] for item in cursor.description] )
    for row in cursor:
      result_rel.append(dict(zip(columns, row)))

/Users/mitch/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:1: DeprecationWarning: context interface will be changed.  Use explicit conn.commit() or conn.rollback().
  if __name__ == '__main__':


In [18]:
print(columns)
print(len(result_rel))

('id', 'relationship_id', 'person_object_id', 'relationship_object_id', 'start_at', 'end_at', 'is_past', 'sequence', 'title', 'created_at', 'updated_at')
402878


In [19]:
result_rel_df = pd.DataFrame(result_rel)
result_rel_df.head(10)

,created_at,end_at,id,is_past,person_object_id,relationship_id,relationship_object_id,sequence,start_at,title,updated_at
0,2007-05-25 07:03:54,None,1,0,p:2,1,c:1,8,None,Co-Founder/CEO/Board of Directors,2013-06-03 09:58:46
1,2007-05-25 07:04:16,None,2,1,p:3,2,c:1,279242,None,VP Marketing,2010-05-21 16:31:34
2,2007-05-25 19:33:03,None,3,0,p:4,3,c:3,4,None,Evangelist,2013-06-29 13:36:58
3,2007-05-25 19:34:53,2009-12-01,4,1,p:5,4,c:3,4,2006-03-01,Senior Director Strategic Alliances,2013-06-29 10:25:34
4,2007-05-25 20:05:33,2010-04-05,6,1,p:7,6,c:4,1,2005-07-01,Chief Executive Officer,2010-04-05 18:41:41
5,2007-05-25 20:06:45,None,7,1,p:8,7,c:4,1,None,Senior Software Engineer,2010-01-12 01:13:29
6,2007-05-25 20:07:50,None,8,1,p:9,8,c:4,1,None,Systems Engineering Manager,2010-08-03 22:00:58
7,2007-05-25 21:51:46,None,9,0,p:10,9,c:5,1,None,"Founder and CEO, Board Of Directors",2010-01-25 21:49:44
8,2007-05-25 22:15:05,None,10,1,p:11,10,c:5,1,None,Co-Founder,2011-08-11 23:48:45
9,2007-05-25 22:15:59,None,11,1,p:12,11,c:5,3,None,"Chief Revenue Officer, VP of Operations",2010-01-25 21:49:45


In [20]:
with conn as cursor:
    cursor.execute('SELECT * FROM cb_milestones')
    fields = [item[0] for item in cursor.description]
    result_mil = []
    columns = tuple( [item[0] for item in cursor.description] )
    for row in cursor:
      result_mil.append(dict(zip(columns, row)))

/Users/mitch/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:1: DeprecationWarning: context interface will be changed.  Use explicit conn.commit() or conn.rollback().
  if __name__ == '__main__':


In [21]:
print(columns)
print(len(result_mil))

('id', 'object_id', 'milestone_at', 'milestone_code', 'description', 'source_url', 'source_description', 'created_at', 'updated_at')
39456


In [22]:
result_mil_df = pd.DataFrame(result_mil)
result_mil_df.head(10)

,created_at,description,id,milestone_at,milestone_code,object_id,source_description,source_url,updated_at
0,2008-06-18 08:14:06,Survives iPhone 3G Stevenote,1,2008-06-09,other,c:12,"Twitter Fails To Fail, Community Rejoices",http://www.techcrunch.com/2008/06/10/twitter-f...,2008-06-18 08:14:06
1,2008-06-18 08:46:28,Twhirl announces support for Seesmic video pla...,2,2008-06-17,other,c:3138,Seesmic Now Available In Twhirl,http://www.inquisitr.com/1103/seesmic-now-avai...,2008-06-18 08:46:28
2,2008-06-18 08:50:24,More than 4 Billion videos viewed at Google Si...,3,2008-06-18,other,c:59,11 Billion Videos Viewed Online in the U.S. in...,http://www.comscore.com/press/release.asp?pres...,2008-06-18 08:50:24
3,2008-06-19 04:14:00,Reddit goes Open Source,4,2008-06-18,other,c:314,reddit goes open source,http://blog.reddit.com/2008/06/reddit-goes-ope...,2008-06-19 04:14:00
4,2008-06-19 04:15:53,Adds the ability to create your own Reddits,5,2008-01-22,other,c:314,Reddit Adds Ability to Create Your Own Reddits,http://www.techcrunch.com/2008/01/22/reddit-ad...,2008-06-19 04:15:53
5,2008-06-19 04:39:01,"Adobe announced new content from MTV Networks,...",6,2008-06-18,other,c:283,None,None,2008-06-19 04:39:01
6,2008-06-19 08:29:10,Closed release of first prototype,7,2008-02-01,other,c:6816,None,None,2008-06-19 16:55:48
7,2008-06-19 08:29:10,Closed release of second prototype,8,2008-05-01,other,c:6816,Diary of a Mobile Internet Startup,http://mobverge.blogspot.com/2008/06/diary-of-...,2008-06-19 16:55:48
8,2008-06-19 15:29:37,"Scoofers, a new search engine that combines se...",9,2008-06-19,other,c:5874,Official launch of Scoofers,http://scoofers.blogspot.com/2008/06/scoofers-...,2008-06-19 19:34:45
9,2008-06-19 17:48:25,Jeremy Zawodny leaves Yahoo for Craigslist.,10,2008-07-01,other,c:2034,I'm Joining Craigslist in July,http://jeremy.zawodny.com/blog/archives/010361...,2008-06-19 17:48:25


In [23]:
result_mil_df[result_mil_df['object_id']=='c:5'].head(10)

,created_at,description,id,milestone_at,milestone_code,object_id,source_description,source_url,updated_at
98,2008-06-27 18:58:14,Facebook adds comments to Mini-Feed,99,2008-06-25,other,c:5,Facebook adds comments to the Mini-Feed. It's ...,http://venturebeat.com/2008/06/25/facebook-add...,2013-01-03 00:36:55
651,2008-10-16 19:43:35,fbFund for Developers awards its grants.,652,2008-10-15,other,c:5,The fbFund Gives Out Cash To 25 Facebook Devel...,http://www.techcrunch.com/2008/10/15/the-fbfun...,2008-10-16 19:43:35
5002,2010-06-22 02:31:39,Facebook Movie Poster Announces 500 Million Fa...,5003,2010-06-19,other,c:5,Facebook Movie Poster Announces 500 Million Fa...,http://techcrunch.com/2010/06/19/facebook-movi...,2010-06-22 02:31:39
5452,2010-07-09 02:30:44,"By the way, Facebook for iPhone has 57.8 milli...",5453,2010-07-08,other,c:5,"By the way, Facebook for iPhone has 57.8 milli...",http://www.businessinsider.com/by-the-way-face...,2010-07-09 02:30:44
5628,2010-07-17 01:38:36,Zuckerberg Makes It Official: Facebook Hits 50...,5629,2010-07-21,other,c:5,Zuckerberg Makes It Official: Facebook Hits 50...,http://techcrunch.com/2010/07/21/facebook-500-...,2010-07-22 02:55:17
6554,2010-08-27 20:07:27,Facebook partners with Y-Combinator,6555,2010-08-27,other,c:5,Facebook To Begin Giving Y Combinator Startups...,http://techcrunch.com/2010/08/26/facebook-y-co...,2010-08-27 20:07:27
7216,2010-09-25 04:16:04,Facebook CEO Mark Zuckerberg Donates $100 Mill...,7217,2010-09-22,other,c:5,Facebook CEO Mark Zuckerberg Donates $100 Mill...,http://www.businessinsider.com/mark-zuckerberg...,2010-09-25 04:16:04
7918,2010-10-22 04:37:41,"Facebook, KPCB Announce $250 Million Social Ne...",7919,2010-10-21,other,c:5,"Facebook, KPCB Announce $250 Million Social Ne...",http://www.businessinsider.com/live-facebook-t...,2010-10-22 04:37:41
8279,2010-11-04 06:20:05,200 Million People Use Facebook Mobile ? Up 3X...,8280,2010-11-03,other,c:5,200 Million People Use Facebook Mobile ? Up 3X...,http://www.businessinsider.com/200-million-peo...,2012-10-10 22:13:30
8495,2010-11-16 04:22:54,Facebook Acquired FB.com From The American Far...,8496,2010-11-15,other,c:5,Facebook Acquired FB.com From The American Far...,http://techcrunch.com/2010/11/15/facebook-acqu...,2010-11-16 04:22:54


In [24]:
with conn as cursor:
    cursor.execute('SELECT * FROM cb_objects')
    fields = [item[0] for item in cursor.description]
    result_obj = []
    columns = tuple( [item[0] for item in cursor.description] )
    for row in cursor:
      result_obj.append(dict(zip(columns, row)))

/Users/mitch/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:1: DeprecationWarning: context interface will be changed.  Use explicit conn.commit() or conn.rollback().
  if __name__ == '__main__':


In [28]:
columns

('id',
 'entity_type',
 'entity_id',
 'parent_id',
 'name',
 'normalized_name',
 'permalink',
 'category_code',
 'status',
 'founded_at',
 'closed_at',
 'domain',
 'homepage_url',
 'twitter_username',
 'logo_url',
 'logo_width',
 'logo_height',
 'short_description',
 'description',
 'overview',
 'tag_list',
 'country_code',
 'state_code',
 'city',
 'region',
 'first_investment_at',
 'last_investment_at',
 'investment_rounds',
 'invested_companies',
 'first_funding_at',
 'last_funding_at',
 'funding_rounds',
 'funding_total_usd',
 'first_milestone_at',
 'last_milestone_at',
 'milestones',
 'relationships',
 'created_by',
 'created_at',
 'updated_at')

In [29]:
print(len(result_obj))

462651


In [33]:
result_obj_df = pd.DataFrame(result_obj)
result_obj_df[['name','id']].head(100)

,name,id
0,Wetpaint,c:1
1,Flektor,c:10
2,There,c:100
3,MYWEBBO,c:10000
4,THE Movie Streamer,c:10001
5,Synergie Media,c:10002
6,Green Basti Ecosystems,c:10003
7,HostNine,c:100042
8,Thomas Publishing,c:10005
9,Vetter Idea Management System,c:100062


In [27]:
# close connection
conn.close()